This is a data cleanup/merge notebook. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
coffee_data = pd.read_csv("coffee_data.csv")
coffee_data.head()

#Clean "Year" column 
coffee_data['YEAR'] = coffee_data['YEAR'].astype(str).str[:-3].astype(np.int64)
coffee_data.head()

,COUNTRY,YEAR,MONTH,TOTAL_PRODUCTION,DOMESTIC_CONSUMPTION,EXPORTABLE_PRODUCTION,GROSS_OPENING_STOCKS
0,Angola,1990,April,50.3450,20.0,30.3450,210.000
1,Bolivia,1990,April,122.7770,25.0,97.7770,54.000
2,Brazil,1990,April,27285.6286,8200.0,19085.6286,26926.000
3,Burundi,1990,April,487.3930,2.0,485.3930,77.000
4,Ecuador,1990,April,1503.8150,350.0,1153.8150,723.963


In [3]:
gdp_data = pd.read_csv("DP_LIVE_23012019025244104.csv")
gdp_data

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDPHRWKD,TOT,USD,A,1970,26.793218,NaN
1,AUS,GDPHRWKD,TOT,USD,A,1971,27.462937,NaN
2,AUS,GDPHRWKD,TOT,USD,A,1972,28.067364,NaN
3,AUS,GDPHRWKD,TOT,USD,A,1973,28.645413,NaN
4,AUS,GDPHRWKD,TOT,USD,A,1974,28.189555,NaN
5,AUS,GDPHRWKD,TOT,USD,A,1975,29.232949,NaN
6,AUS,GDPHRWKD,TOT,USD,A,1976,29.720941,NaN
7,AUS,GDPHRWKD,TOT,USD,A,1977,29.687325,NaN
8,AUS,GDPHRWKD,TOT,USD,A,1978,30.939895,NaN
9,AUS,GDPHRWKD,TOT,USD,A,1979,31.089906,NaN


In [4]:
cc_data = pd.read_csv("country_code_web.csv", encoding="latin")
cc_data.head()

,Country or area name,ISO ALPHA-3 code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [5]:
new_cc_data = cc_data.rename(columns={"Country or area name":"COUNTRY","ISO ALPHA-3 code":"LOCATION" })
new_cc_data.head()

,COUNTRY,LOCATION
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [6]:
combined_coffee_country_df = pd.merge(coffee_data, new_cc_data, on="COUNTRY")
combined_coffee_country_df

,COUNTRY,YEAR,MONTH,TOTAL_PRODUCTION,DOMESTIC_CONSUMPTION,EXPORTABLE_PRODUCTION,GROSS_OPENING_STOCKS,LOCATION
0,Angola,1990,April,50.3450,20.0,30.3450,210.0,AGO
1,Angola,1991,April,79.3310,30.0,49.3310,161.0,AGO
2,Angola,1992,April,77.5200,35.0,42.5200,136.0,AGO
3,Angola,1993,April,32.6080,20.0,12.6080,111.0,AGO
4,Angola,1994,April,76.8020,25.0,51.8020,96.0,AGO
5,Angola,1995,April,62.1090,10.0,52.1090,36.0,AGO
6,Angola,1996,April,70.9250,20.0,50.9250,40.0,AGO
7,Angola,1997,April,64.3300,40.0,24.3300,40.0,AGO
8,Angola,1998,April,85.3440,30.0,55.3440,10.0,AGO
9,Angola,1999,April,54.9390,20.0,34.9390,10.0,AGO


In [7]:
#all_combined = pd.merge(combined_coffee_country_df, gdp_data, on="LOCATION")
#all_combined